<a href="https://colab.research.google.com/github/amengemeda/ISproject-2/blob/main/Amharic_Hate_Speech_detection_using_mBERT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#installing the transformers package
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
#installing the datasets package
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.2 MB/s 
     |████████████████████████████████| 212 kB 62.7 MB/s 
     |████████████████████████████████| 115 kB 63.8 MB/s 
     |████████████████████████████████| 127 kB 68.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [5]:
#importing the datasets package
from datasets import Dataset
import datasets
#import load metric for model evaluation
from datasets import load_metric

In [6]:
#import numpy and pandas for mathematical computation and data manipulation respectively 
import numpy as np
import pandas as pd
#import drive package to connect this colab file with the drive where the data will be retrived from
from google.colab import drive
#import the pipeline of transformers
from transformers import pipeline
#import AutoTokenizer for tokenization purposes
from transformers import AutoTokenizer


In [7]:
#import torch
import torch
#import Data loader from torch
from torch.utils.data import DataLoader
#import an optimizer
from torch.optim import AdamW
#import tqdm for a progress bar
from tqdm.auto import tqdm

In [8]:
#mount google drive to access the dataset directly from the drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
#fetch the dataset from the drive
Labels=pd.read_csv('/content/drive/MyDrive/Machine Learning/Data/Amharic-Hate-Speech-Dataset/Labels.txt',header=None)
Posts=pd.read_csv('/content/drive/MyDrive/Machine Learning/Data/Amharic-Hate-Speech-Dataset/Posts.txt',header=None)

In [10]:
#naming the columns
Labels.columns = ["labels"]
Posts.columns = ["post"]

In [11]:
#encoding the classes into numerical data
Labels = Labels.replace(['Free', 'Free ','Hate'],[0,0,1]) 

In [12]:
#check the encoded label data
Labels.head(10)

,labels
0,0
1,0
2,0
3,0
4,1
5,0
6,0
7,0
8,1
9,0


In [13]:
#check the Amharic data
Posts.head(1000)

,post
0,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደ...
2,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...
3,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4,ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...
...,...
995,መፈንቅለመንግስት ከሽፏልእንዴ የየክልሉ ህዝብ ምን ይጠብቃል ሆ ብሎ ሄዶ ...
996,ሰው በሩን እንደፈለገ መክፈት እንደፈለገ መዝጋት ይችላል የምን አውቅልሻ...
997,ክክክክክ የሱዳን ዜጋ ናችሁ እንዴ ባለ ነጠላ ጎጃሜ ሁላ
998,የምን ማለቃቀስ ነው ምድረ ፎካሪ ሁላ


In [14]:
#merge the datasets
Frames = [Labels, Posts]
Merged = pd.concat(Frames, axis=1)


In [15]:
#preview of merged data
Merged

,labels,post
0,0,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1,0,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደ...
2,0,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...
3,0,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4,1,ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...
...,...,...
29995,1,በአሉ የሁሉም ኢትዮጵያዊ ስላልሆነ በኦሮምኛው ቢለፋደድ ምን አገባን
29996,0,ተባረክ አብቹ ፈር ቀዳጅ ስለሆንህ መጋረጃው መቀደድ ስለጀመረ
29997,0,እስከ አሁን አንተ ብቻ ነው በ መፅሀፍ ያልቻልከው አንተም ታሪክ እ...
29998,1,ህገወጥት ጠቅላይ ሚንስትር ፅቤት የተፈቀደ ሆኖ ህዝብን እንዴት ህግ አክ...


In [16]:
#Divide the dataset into train and test categories 
msk = np.random.rand(len(Merged)) < 0.8
train_dataset = Merged[msk]
test_dataset = Merged[~msk]

In [17]:
#convert format of the dataset to HuggingFace Dataset from Pandas DataFrame
test_dataset=Dataset.from_pandas(test_dataset)

In [18]:
#convert the format of the dataset to HuggingFace Dataset from Pandas DataFrame
train_dataset=Dataset.from_pandas(train_dataset)

In [19]:
#preview of the dataset after conversion
(test_dataset)

Dataset({
    features: ['labels', 'post', '__index_level_0__'],
    num_rows: 6078
})

In [20]:
#remove unnecessary column
test_dataset=test_dataset.remove_columns("__index_level_0__")
train_dataset=train_dataset.remove_columns("__index_level_0__")

In [21]:
#combine the train and test dataset into one datset
main_dataset= datasets.DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

In [22]:
#preview of the dataset after merging
main_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'post'],
        num_rows: 23922
    })
    test: Dataset({
        features: ['labels', 'post'],
        num_rows: 6078
    })
})

In [23]:
# training and testing data size
training_data_size = main_dataset['train'].num_rows
testing_data_size = main_dataset['test'].num_rows

In [24]:
#loading a tokenizer from the pretrained model
tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-amharic")

In [25]:
#Have a tokenizer function that uses the tokenizer 
def tokenize_function(data):
    return tokenizer(data["post"], padding="max_length", truncation=True)

In [26]:
#Tokenize all the data using the mapping functionality
tokenized_datasets = main_dataset.map(tokenize_function)


  0%|          | 0/23922 [00:00<?, ?ex/s]

  0%|          | 0/6078 [00:00<?, ?ex/s]

In [27]:
#empty cache
torch.cuda.empty_cache()

In [28]:
#remove the posts column as it is no longer needed
tokenized_datasets = tokenized_datasets.remove_columns(["post"])

In [29]:
# tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

In [30]:
#changing the format of the tokenized dataset to torch
tokenized_datasets.set_format("torch")

In [31]:
#shuffeling and selecting the needed size of dataset for training and evaluating the model
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(training_data_size))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(testing_data_size))

In [32]:
# preview of the shuffeled and selected evaluation dataset
small_eval_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6078
})

In [33]:
# preview of the shuffeled and selected training dataset
small_train_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 23922
})

In [34]:
#load the dataset using DataLoader
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [35]:
#Load auto mode classifier from the pretrained model 
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-amharic", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("Davlan/", num_labels=2)

Some weights of the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-amharic were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model check

In [36]:
#load an optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

In [37]:
#import a scheduler
from transformers import get_scheduler

In [47]:
#hyperparameter
#set number of epochs
num_epochs = 10
num_training_steps = num_epochs * len(train_dataloader)

In [48]:
#set up the scheduler
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [49]:
# try to check and load if there is a GPU. If not, revert back to CPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [50]:
#create a progress bar to keep track of the progress of the model training
training_progress_bar = tqdm(range(num_training_steps))

  0%|          | 0/29910 [00:00<?, ?it/s]

In [51]:
#train model
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        training_progress_bar.update(1)

In [52]:
#number of evaluation steps
num_evaluation_steps = len(eval_dataloader)

In [59]:
#load the F1 evaluation metric
metric = load_metric('f1')

In [60]:
#create a progress bar to keep track of the progress of the model evaluation
evaluation_progress_bar = tqdm(range(num_evaluation_steps))

  0%|          | 0/760 [00:00<?, ?it/s]

In [61]:
#evaluate model
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])
    evaluation_progress_bar.update(1)

metric.compute()

{'f1': 0.6771139405811296}

In [62]:
#install huggingface_hub package to interact with huggingface platform
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [63]:
from huggingface_hub import notebook_login

In [64]:
#login to huggingface
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [65]:
#specify the path for the trainde model and tokenizer to huggingface repository
model.save_pretrained("path/to/improved-amharic-hate-speech-detection")
tokenizer.save_pretrained("path/to/improved-amharic-hate-speech-detection")

('path/to/improved-amharic-hate-speech-detection/tokenizer_config.json',
 'path/to/improved-amharic-hate-speech-detection/special_tokens_map.json',
 'path/to/improved-amharic-hate-speech-detection/vocab.txt',
 'path/to/improved-amharic-hate-speech-detection/added_tokens.json',
 'path/to/improved-amharic-hate-speech-detection/tokenizer.json')

In [66]:
#push the trained model to huggingface repository
model.push_to_hub("improved-amharic-hate-speech-detection")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:42: FutureWarning: Deprecated positional argument(s) used in 'create_repo': pass token='improved-amharic-hate-speech-detection' as keyword args. From version 0.12 passing these as positional arguments will result in an error,
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:684: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/amengemeda/improved-amharic-hate-speech-detection into local empty directory.


Upload file pytorch_model.bin:   0%|          | 3.34k/679M [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/amengemeda/improved-amharic-hate-speech-detection
   8b84a5a..7f2e039  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/amengemeda/improved-amharic-hate-speech-detection
   8b84a5a..7f2e039  main -> main



'https://huggingface.co/amengemeda/improved-amharic-hate-speech-detection/commit/7f2e03993cf5129b630bda02b200cc74fecb303f'

In [67]:
#push the tokenizer to huggingface repository
tokenizer.push_to_hub("improved-amharic-hate-speech-detection")

Adding files tracked by Git LFS: ['tokenizer.json', 'vocab.txt']. This may take a bit of time if the files are large.


Upload file tokenizer.json:   0%|          | 3.34k/3.32M [00:00<?, ?B/s]

Upload file vocab.txt:   0%|          | 3.34k/1.48M [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/amengemeda/improved-amharic-hate-speech-detection
   7f2e039..55eb209  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/amengemeda/improved-amharic-hate-speech-detection
   7f2e039..55eb209  main -> main



'https://huggingface.co/amengemeda/improved-amharic-hate-speech-detection/commit/55eb209f7297fa0f83f4a62e6572c4c9fb2a11ff'